In [12]:
import json
import random
with open("data/imdb_train.json") as f:
    data=json.load(f)
random.shuffle(data) 
print(data[0])

# We need to gather the texts, into a list
texts=[one_example["text"] for one_example in data]
labels=[one_example["class"] for one_example in data]
print(texts[:2])
print(labels[:2])

{'class': 'pos', 'text': 'In the early 1970s, many of us who had embraced hippy/acid/alternative culture wholeheartedly, had realised that mainstream society was not going to change in the way we thought it would. For me this film defined the tension of \\now what?\\" that many of the people I knew felt at that time. Do we head for the commune and create our own vision of utopia or do we radicalize and push for change?  From the futile gestures to police, the radical rhetoric, the music of Pink Floyd, to the love-making and the explosive images in the desert - so much is in there. It captures well a significant moment in time."'}
['In the early 1970s, many of us who had embraced hippy/acid/alternative culture wholeheartedly, had realised that mainstream society was not going to change in the way we thought it would. For me this film defined the tension of \\now what?\\" that many of the people I knew felt at that time. Do we head for the commune and create our own vision of utopia or d

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy
analyzer=CountVectorizer(lowercase=False).build_analyzer() # includes tokenizer and preprocessing
print(analyzer(texts[0]))


#vectorizer=vectorizer.fit(texts) # learns a vocabulary dictionary
#print("Vocabulary size:",len(vectorizer.vocabulary_))
#print("First 5 items in the vocabulary:",list(vectorizer.vocabulary_.keys())[:5])
#print("How many words are recognized from the data:",numpy.count_nonzero(vectorizer.transform(["i went today to new_york"]).todense()))

['In', 'the', 'early', '1970s', 'many', 'of', 'us', 'who', 'had', 'embraced', 'hippy', 'acid', 'alternative', 'culture', 'wholeheartedly', 'had', 'realised', 'that', 'mainstream', 'society', 'was', 'not', 'going', 'to', 'change', 'in', 'the', 'way', 'we', 'thought', 'it', 'would', 'For', 'me', 'this', 'film', 'defined', 'the', 'tension', 'of', 'now', 'what', 'that', 'many', 'of', 'the', 'people', 'knew', 'felt', 'at', 'that', 'time', 'Do', 'we', 'head', 'for', 'the', 'commune', 'and', 'create', 'our', 'own', 'vision', 'of', 'utopia', 'or', 'do', 'we', 'radicalize', 'and', 'push', 'for', 'change', 'From', 'the', 'futile', 'gestures', 'to', 'police', 'the', 'radical', 'rhetoric', 'the', 'music', 'of', 'Pink', 'Floyd', 'to', 'the', 'love', 'making', 'and', 'the', 'explosive', 'images', 'in', 'the', 'desert', 'so', 'much', 'is', 'in', 'there', 'It', 'captures', 'well', 'significant', 'moment', 'in', 'time']


In [14]:
from gensim.models import KeyedVectors

vector_model=KeyedVectors.load_word2vec_format("data/GoogleNews-vectors-negative300.bin", binary=True, limit=50000)

# sort based on the index to make sure they are in the correct order
# lowercase everything because that is the dafeault setting in CountVectorizer
words=[k for k,v in sorted(vector_model.vocab.items(), key=lambda x:x[1].index)]
print("Words from embedding model:",len(words))
print("First 50 words:",words[:50])

Words from embedding model: 50000
First 50 words: ['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are', 'I', 'have', 'he', 'will', 'has', '####', 'his', 'an', 'this', 'or', 'their', 'who', 'they', 'but', '$', 'had', 'year', 'were', 'we', 'more', '###', 'up', 'been', 'you', 'its', 'one', 'about', 'would', 'which', 'out']


In [15]:
# init the vectorizer vocabulary using words from the embedding model

def init_vocabulary(vocab, text, text_analyzer):
    for word in analyzer(text):
        vocab.setdefault(word, len(vocab))
    return vocab

words_txt=" ".join(words)
vocabulary={"<SPECIAL>": 0} # zero has a special meaning, prevent using it for a normal word
vocabulary=init_vocabulary(vocabulary,words_txt,analyzer)
print("Words from embedding model:",len(vocabulary))
#for word in words:
#    if word not in vocab:
#        print(word)
#    vocab.setdefault(word, len(vectorizer.vocabulary_)) # setdefault adds the word if it does not already exist
#vectorizer.vocabulary_=vocab
#print("Vocabulary size:",len(vectorizer.vocabulary_))
#print("First 5 items in the vocabulary:",list(vectorizer.vocabulary_.keys())[:5])
#print("How many words are recognized from the data:",numpy.count_nonzero(vectorizer.transform(["i went today to new_york"]).todense()))

Words from embedding model: 49412


In [16]:
def vectorizer(vocab, texts):
    vectorized_data=[] # turn text into numbers based on our vocabulary mapping
    for one_example in texts:
        vectorized_example=[]
        for word in analyzer(one_example):
            vocab.setdefault(word, len(vocab)) # add word to out vocabulary if it does not exist
            vectorized_example.append(vocab[word])
        vectorized_data.append(vectorized_example)
    
    vectorized_data=numpy.array(vectorized_data) # turn python list into numpy matrix
    return vectorized_data

vectorized_data=vectorizer(vocabulary, texts)

# now vectorized data is in the same as feature_matrix, but in slightly different format
print("Words in vocabulary:",len(vocabulary))
print("Vectorized data shape:",vectorized_data.shape)
print("Firs example vectorized:",vectorized_data[0])
        

Words in vocabulary: 111151
Vectorized data shape: (25000,)
Firs example vectorized: [65, 10, 262, 49412, 122, 49413, 152, 28, 31, 8427, 49414, 14932, 2116, 1983, 31886, 31, 49415, 3, 5220, 1928, 9, 12, 116, 49416, 399, 1, 10, 123, 34, 542, 14, 42, 178, 157, 25, 659, 3634, 10, 6551, 49413, 90, 76, 3, 122, 49413, 10, 63, 1001, 1002, 11, 3, 54, 813, 34, 349, 2, 10, 46989, 49417, 778, 66, 231, 2602, 49413, 49418, 26, 53, 34, 49419, 49417, 1634, 2, 399, 1202, 10, 20306, 14409, 49416, 174, 10, 5975, 8014, 10, 603, 49413, 15059, 8931, 49416, 10, 713, 297, 49417, 10, 6733, 2589, 1, 10, 6667, 78, 139, 4, 1, 67, 46, 12679, 104, 789, 1282, 1, 54]


In [17]:
#feature_matrix=vectorizer.transform(texts)
#print(feature_matrix.shape)

# labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder=LabelEncoder() #Turns class labels into integers
one_hot_encoder=OneHotEncoder(sparse=False) #Turns class integers into one-hot encoding
class_numbers=label_encoder.fit_transform(labels)
print("class_numbers shape=",class_numbers.shape)
print("class_numbers",class_numbers)
print("class labels",label_encoder.classes_)
#And now yet the one-hot encoding
classes_1hot=one_hot_encoder.fit_transform(class_numbers.reshape(-1,1))
print("classes_1hot",classes_1hot)


class_numbers shape= (25000,)
class_numbers [1 0 1 ... 1 1 1]
class labels ['neg' 'pos']
classes_1hot [[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


## Network

In [18]:

def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    import string
    pretrained_embeddings=numpy.zeros((len(vocab),embedding_model.vectors.shape[1])) # initialize new matrix (words x embedding dim)
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)
print("Shape of pretrained embeddings:",pretrained.shape)


Found pretrained vectors for 48923 words.
Shape of pretrained embeddings: (111151, 300)


In [29]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Activation
from keras import backend as K
from keras.layers.core import Lambda

from keras.preprocessing.sequence import pad_sequences



vectorized_data_padded=pad_sequences(vectorized_data)#, maxlen=100, padding='post', truncating='post')
print("New shape:", vectorized_data_padded.shape)
example_count,sequence_len=vectorized_data_padded.shape
example_count,class_count=classes_1hot.shape

#example_count,feature_count=feature_matrix.shape

#example_count,class_count=classes_1hot.shape


inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary),200,mask_zero=True)(inp)
sums=Lambda(lambda s: K.sum(s, axis=1), output_shape=lambda s: (s[0],s[2]))(embeddings) # custom layer to sum all embeddings
tanh=Activation("tanh")(sums)

outp=Dense(class_count,activation="softmax")(tanh)
model=Model(inputs=[inp], outputs=[outp])
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

print(model.summary())

hist=model.fit(vectorized_data_padded,classes_1hot,batch_size=100,verbose=1,epochs=1,validation_split=0.1)

New shape: (25000, 2366)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 2366)              0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 2366, 200)         22230200  
_________________________________________________________________
lambda_5 (Lambda)            (None, 200)               0         
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402       
Total params: 22,230,602
Trainable params: 22,230,602
Non-trainable params: 0
_________________________________________________________________
None
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [============================